In [1]:
import os
import glob
from pci.fimport import fimport
from pci.atcor import atcor
from pci.mosprep import mosprep
from pci.mosaic import mosaic
from pci.pcimod import pcimod

In [4]:
def get_band_paths(current_directory):
    """Get paths for bands 4, 3, and 2 from a single-band image folder."""
    bands = {4: None, 3: None, 2: None}
    for band_num in bands.keys():
        band_path = glob.glob(os.path.join(current_directory, f"**/*_B{band_num}.TIF"), recursive=True)
        if band_path:
            bands[band_num] = band_path[0]
    return bands

In [5]:
# Define directories
parent_directory = os.path.join(os.getcwd(), 'images')
image_folders = os.listdir(parent_directory)

# Store corrected image paths
corrected_images = []

for image_folder in image_folders:
    current_directory = os.path.join(parent_directory, image_folder)
    band_paths = get_band_paths(current_directory)

    if all(band_paths.values()):  # Ensure all bands exist
        output_file = os.path.join(current_directory, f"{image_folder}_corrected.pix")
        
        # Import to PCI PIX format
        fimport(fili=list(band_paths.values()), filiop=[], dbiw=[], dbic=[], dbib=[], 
                ftype="TIF", foptions=[], dbow=[], dboc=[], dbob=[], filo=output_file)

        # Apply Atmospheric Correction
        atcor(fili=output_file, dbic=[1, 2, 3], sensor="Landsat8", filo=output_file, ftype="PIX")

        corrected_images.append(output_file)

In [ ]:
# Mosaic the corrected images
if corrected_images:
    mosaic_output = os.path.join(parent_directory, "mosaic_output.pix")
    
    # Prepare mosaic
    mosprep(fili=corrected_images, filo=mosaic_output, ftype="PIX")
    
    # Perform mosaic
    mosaic(fili=corrected_images, dbic=[1, 2, 3], filo=mosaic_output, ftype="PIX")

    print("Mosaic completed:", mosaic_output)

    # Export cutlines as a shapefile
    cutline_shapefile = os.path.join(parent_directory, "mosaic_cutlines.shp")
    pcimod(fili=mosaic_output, dbic=[], dbib=[], dbiw=[], dboc=[], dbob=[], dbow=[],
           ftype="SHAPE", filo=cutline_shapefile)

    print("Cutlines exported successfully:", cutline_shapefile)
else:
    print("No valid images found for processing.")

In [ ]:
"""
Purpose: This script processes Landsat 8 imagery by importing single-band images, applying atmospheric correction,
         creating a seamless mosaic, and exporting the mosaic cutlines as a shapefile.
Author: Biplap KC
Course: ADIP
Assignment: MOSAIC
"""

import os
import glob
import time
from datetime import timedelta
from pci.fimport import fimport
from pci.atcor import atcor
from pci.mosprep import mosprep
from pci.mosaic import mosaic
from pci.pcimod import pcimod

# Start timer
start_time = time.time()

# Configure logging
print("Script started. Setting up directories and checking inputs...")

# Define directories
parent_directory = os.path.abspath(os.path.join(os.getcwd(), 'images'))
output_directory = os.path.abspath(os.path.join(os.getcwd(), 'output'))

# Create output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
    print(f"Created output directory: {output_directory}")

# Check if the parent directory exists
if not os.path.exists(parent_directory):
    print(f"Error: Parent directory '{parent_directory}' does not exist.")
    exit(1)

# Get list of image folders
image_folders = [f for f in os.listdir(parent_directory) if os.path.isdir(os.path.join(parent_directory, f))]

if not image_folders:
    print("Error: No image folders found in the parent directory.")
    exit(1)

def get_band_paths(current_directory):
    """
    Get paths for bands 4, 3, and 2 from a single-band image folder.
    Args:
        current_directory (str): Path to the folder containing single-band images.
    Returns:
        dict: A dictionary with band numbers as keys and file paths as values.
    """
    bands = {4: None, 3: None, 2: None}  # Bands 4, 3, and 2 for Landsat 8
    for band_num in bands.keys():
        band_path = glob.glob(os.path.join(current_directory, f"**/*_B{band_num}.TIF"), recursive=True)
        if band_path:
            bands[band_num] = band_path[0]
    return bands if all(bands.values()) else None  # Ensure all bands exist

def process_image_folder(image_folder, parent_directory, output_directory):
    """
    Process an image folder by importing bands, applying atmospheric correction, and returning the output file.
    Args:
        image_folder (str): Name of the image folder.
        parent_directory (str): Path to the parent directory containing image folders.
        output_directory (str): Path to the output directory.
    Returns:
        str: Path to the corrected output file.
    """
    current_directory = os.path.join(parent_directory, image_folder)
    band_paths = get_band_paths(current_directory)

    if not band_paths:
        print(f"Warning: Skipping {image_folder} due to missing bands. Expected bands: 4, 3, 2.")
        return None

    output_file = os.path.join(output_directory, f"{image_folder}_corrected.pix")
    
    # Import bands to PCI PIX format
    print(f"Importing bands for {image_folder}...")
    fimport(fili=list(band_paths.values()), filo=output_file, ftype="TIF")
    
    # Apply atmospheric correction
    print(f"Applying atmospheric correction for {image_folder}...")
    atcor(fili=output_file, dbic=[1, 2, 3], sensor="Landsat8", filo=output_file, ftype="PIX")
    
    print(f"Completed processing for {image_folder}.")
    return output_file

# Store corrected image paths
corrected_images = []

# Process each image folder
for image_folder in image_folders:
    output_file = process_image_folder(image_folder, parent_directory, output_directory)
    if output_file:
        corrected_images.append(output_file)

# Mosaic the corrected images
if corrected_images:
    mosaic_output = os.path.join(output_directory, "mosaic_output.pix")
    
    # Prepare and perform mosaic
    print("Preparing mosaic...")
    mosprep(fili=corrected_images, filo=mosaic_output, ftype="PIX")
    
    print("Creating seamless mosaic...")
    mosaic(fili=corrected_images, dbic=[1, 2, 3], filo=mosaic_output, ftype="PIX")
    print(f"Mosaic completed: {mosaic_output}")

    # Export cutlines as a shapefile
    cutline_shapefile = os.path.join(output_directory, "mosaic_cutlines.shp")
    print("Exporting mosaic cutlines as a shapefile...")
    pcimod(fili=mosaic_output, dbic=[1], filo=cutline_shapefile, ftype="SHAPE")
    print(f"Cutlines exported successfully: {cutline_shapefile}")
else:
    print("Error: No valid images found for processing.")

# Calculate and print script runtime
end_time = time.time()
runtime = timedelta(seconds=end_time - start_time)
print(f"Script completed in {runtime}.")